In [1]:
from gerrychain import Graph

In [2]:
# Read New Mexico county graph from the json file "OK_county.json"
filename = 'NM_county.json'

# GerryChain has a built-in function for reading graphs of this type:
G = Graph.from_json( filename )

In [3]:
# Let's impose a 1% population deviation (+/- 0.5%)
deviation = 0.01

import math
k = 3          # number of districts
total_population = sum(G.nodes[node]['TOTPOP'] for node in G.nodes)

L = math.ceil((1-deviation/2)*total_population/k)
U = math.floor((1+deviation/2)*total_population/k)
print("Using L =",L,"and U =",U,"and k =",k)

Using L = 682962 and U = 689824 and k = 3


In [4]:
# A nice ordering of New Mexico's vertices (counties)
ordering = [18, 19, 26, 22, 14, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 20, 21, 23, 24, 25, 27, 28, 29, 30, 31, 32]

In [5]:
import gurobipy as gp
from gurobipy import GRB

# create model 
m = gp.Model()

# create variables
m._X = m.addVars(G.nodes, k, vtype=GRB.BINARY) # x[i,j] equals one when county i is assigned to district j

Academic license - for non-commercial use only - expires 2021-06-27
Using license file C:\Users\buchanan\gurobi.lic


In [6]:
# null objective 
m.setObjective( 0.0, GRB.MINIMIZE )

In [7]:
# add constraints saying that each county i is assigned to one district
m.addConstrs( gp.quicksum(m._X[i,j] for j in range(k)) == 1 for i in G.nodes)

# add constraints saying that each district has population at least L and at most U
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * m._X[i,j] for i in G.nodes) >= L for j in range(k) )
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * m._X[i,j] for i in G.nodes) <= U for j in range(k) )

m.update()

In [8]:
# separation routines
import networkx as nx

def find_fischetti_separator(DG, component, b):
    neighbors_component = [False for i in DG.nodes]
    for i in nx.node_boundary(DG, component, None):
        neighbors_component[i] = True
    
    visited = [False for i in DG.nodes]
    child = [b]
    visited[b] = True
    
    while child:
        parent = child
        child = []
        for i in parent:
            if not neighbors_component[i]:
                for j in DG.neighbors(i):
                    if not visited[j]:
                        child.append(j)
                        visited[j] = True
    
    C = [ i for i in DG.nodes if neighbors_component[i] and visited[i] ]
    return C


def lcut_separation_generic(m, where):
    if where == GRB.Callback.MIPSOL:
        xval = m.cbGetSolution(m._X)
        DG = m._DG
        U = m._U
        population = m._population
        k = m._k
        
        for j in range(k):
            
            # vertices assigned to this district (label j)
            S = [v for v in DG.nodes if xval[v,j] > 0.5]
            
            # what shall we deem as the "root" of this district? call it b
            max_cp = 0
            max_component = []

            for component in nx.strongly_connected_components(DG.subgraph(S)):
                if max_cp < sum(population[v] for v in component):
                    max_cp = sum(population[v] for v in component)
                    max_component = component

            # find some vertex "b" that has largest population in this component
            maxpb = max(population[v] for v in max_component)
            maxpb_vertices = [ v for v in max_component if population[v] == maxpb ]
            b = maxpb_vertices[0]  
            
            for component in nx.strongly_connected_components(DG.subgraph(S)):
                
                if b in component: 
                    continue
                
                # find some vertex "a" that has largest population in this component
                maxp = max(population[v] for v in component)
                maxp_vertices = [ v for v in component if population[v] == maxp ]
                a = maxp_vertices[0]  
                    
                # get minimal a,b-separator
                C = find_fischetti_separator(DG, component, b)
                
                # make it a minimal *length-U* a,b-separator
                for (u,v) in DG.edges():
                    DG[u][v]['lcutweight'] = population[u]   
                    
                # "remove" C from graph
                for c in C:
                    for node in DG.neighbors(c):
                        DG[c][node]['lcutweight'] = U+1
                
                # is C\{c} a length-U a,b-separator still? If so, remove c from C
                drop_from_C = []
                for c in C:
                    
                    # temporarily add c back to graph (i.e., "remove" c from cut C)
                    for node in DG.neighbors(c):
                        DG[c][node]['lcutweight'] = population[c]
                    
                    # what is distance from a to b in G-C ?
                    distance_from_a = nx.single_source_dijkstra_path_length(DG, a, weight='lcutweight')
                    
                    if distance_from_a[b] + population[b] > U:
                        # c was not needed in the cut C. delete c from C
                        drop_from_C.append(c)
                    else:
                        # keep c in C. revert arc weights back to "infinity"
                        for node in DG.neighbors(c):
                            DG[c][node]['lcutweight'] = U+1
                    
                # add lazy cut
                minC = [c for c in C if c not in drop_from_C ]
                
                m.cbLazy( m._X[a,j] + m._X[b,j] <= 1 + gp.quicksum(m._X[c,j] for c in minC) )

In [9]:
# set LCUT callback
m.Params.lazyConstraints = 1
m._population = [G.nodes[i]['TOTPOP'] for i in G.nodes]
m._k = k
m._U = U
m._DG = nx.DiGraph(G) # bidirected version of G
m._callback = lcut_separation_generic 

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0


In [10]:
# Do some (safe) fixing to avoid symmetric solutions

# fix the first vertex in the ordering to first district
m._X[ordering[0],0].lb = 1

# fix the second vertex in the ordering to the second district
m._X[ordering[1],1].lb = 1

# Find the N best solutions
N = 1000
m.Params.PoolSearchMode = 2 # finds N best solutions
m.Params.PoolSolutions = N

# solve IP model
m.optimize(m._callback)

Changed value of parameter PoolSearchMode to 2
   Prev: 0  Min: 0  Max: 2  Default: 0
Changed value of parameter PoolSolutions to 1000
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 39 rows, 99 columns and 297 nonzeros
Model fingerprint: 0x5dfc4775
Variable types: 0 continuous, 99 integer (99 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+05]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+05]
Presolve removed 16 rows and 34 columns
Presolve time: 0.00s
Presolved: 23 rows, 65 columns, 209 nonzeros
Variable types: 0 continuous, 65 integer (65 binary)

Root relaxation: objective 0.000000e+00, 27 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

 

In [11]:
print("The number of cut edges is",m.objval)

# retrieve the districts and their populations
districts = [ [i for i in G.nodes if m._X[i,j].x > 0.5] for j in range(k)]
district_counties = [ [ G.nodes[i]["NAME10"] for i in districts[j] ] for j in range(k)]
district_populations = [ sum(G.nodes[i]["TOTPOP"] for i in districts[j]) for j in range(k) ]

# print district info
for j in range(k):
    print("District",j,"has population",district_populations[j],"and contains counties",district_counties[j])

The number of cut edges is 0.0
District 0 has population 685656 and contains counties ['Guadalupe', 'Torrance', 'De Baca', 'Bernalillo']
District 1 has population 687790 and contains counties ['Grant', 'Hidalgo', 'Cibola', 'Los Alamos', 'Doña Ana', 'Valencia', 'Catron', 'Sandoval', 'Santa Fe', 'Socorro', 'Luna']
District 2 has population 685733 and contains counties ['Harding', 'Sierra', 'Lea', 'Otero', 'San Juan', 'Roosevelt', 'Curry', 'Taos', 'Eddy', 'Quay', 'Colfax', 'Chaves', 'Rio Arriba', 'San Miguel', 'Lincoln', 'McKinley', 'Mora', 'Union']


In [ ]:
# Let's draw it on a map
import geopandas as gpd

In [ ]:
# Read New Mexico county shapefile from "NM_county.shp"
filename = 'NM_county.shp'

# Read geopandas dataframe from file
df = gpd.read_file( filename )

In [ ]:
import matplotlib.pyplot as plt

def export_to_png(G, df, districts, filename):
    
    assignment = [ -1 for u in G.nodes ]
    
    for j in range(len(districts)):
        for i in districts[j]:
            geoID = G.nodes[i]["GEOID10"]
            for u in G.nodes:
                if geoID == df['GEOID10'][u]:
                    assignment[u] = j
    
    if min(assignment[v] for v in G.nodes) < 0:
        print("Error: did not assign all nodes in district map png.")
    else:
        df['assignment'] = assignment
        my_fig = df.plot(column='assignment').get_figure()
        RESIZE_FACTOR = 3
        my_fig.set_size_inches(my_fig.get_size_inches()*RESIZE_FACTOR)
        plt.axis('off')
        my_fig.savefig(filename)
        plt.close(my_fig)

In [ ]:
# retrieve and map each solution

all_plans = list()

for sol in range(m.SolCount):
    m.Params.SolutionNumber = sol
    districts = [ [i for i in G.nodes if m._X[i,j].Xn > 0.5] for j in range(k) ]
    
    filename = 'NM-' + str(f"{sol+1:04d}") + '.png'
    export_to_png(G, df, districts, filename)
    
    all_plans.append(districts)
    
print(all_plans)
print("Number of plans =",len(all_plans))